In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from pandas.tseries.holiday import USFederalHolidayCalendar

In [2]:
holdings = pd.read_csv('raw_data/holdings_without_na.csv')

C:\Users\ohste\AppData\Local\Temp\ipykernel_16156\2931917762.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  holdings = pd.read_csv('raw_data/holdings_without_na.csv')


In [3]:
print(f"Number of rows of all holdings: {len(holdings)}")

Number of rows of all holdings: 108529512


# Remove filings after filing deadline

In [4]:
holdings["date_filed"] = pd.to_datetime(holdings["date_filed"])
holdings["date_holding"] = pd.to_datetime(holdings["date_holding"])

In [5]:
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2000-01-01', end='2022-12-31').to_pydatetime()

# If deadline falls on Sat, Sun or Holiday, move to next business day
def get_filing_deadline(quarter_end):
    curr_deadline = quarter_end + datetime.timedelta(days=45)
    while curr_deadline.weekday() >= 5 or curr_deadline in holidays:
        curr_deadline = curr_deadline + datetime.timedelta(days=1)
    return curr_deadline

In [6]:
filing_deadline_map = {}
for quarter_end in holdings["date_holding"].drop_duplicates():
    filing_deadline_map[quarter_end] = get_filing_deadline(quarter_end)

In [7]:
filing_deadline_map

{Timestamp('1999-12-31 00:00:00'): Timestamp('2000-02-14 00:00:00'),
 Timestamp('1999-03-31 00:00:00'): Timestamp('1999-05-17 00:00:00'),
 Timestamp('1999-09-30 00:00:00'): Timestamp('1999-11-15 00:00:00'),
 Timestamp('1999-12-30 00:00:00'): Timestamp('2000-02-14 00:00:00'),
 Timestamp('1999-12-01 00:00:00'): Timestamp('2000-01-18 00:00:00'),
 Timestamp('1999-06-30 00:00:00'): Timestamp('1999-08-16 00:00:00'),
 Timestamp('2000-02-11 00:00:00'): Timestamp('2000-03-27 00:00:00'),
 Timestamp('2000-02-14 00:00:00'): Timestamp('2000-03-30 00:00:00'),
 Timestamp('1999-12-03 00:00:00'): Timestamp('2000-01-18 00:00:00'),
 Timestamp('2000-02-29 00:00:00'): Timestamp('2000-04-14 00:00:00'),
 Timestamp('2000-03-31 00:00:00'): Timestamp('2000-05-15 00:00:00'),
 Timestamp('2000-04-06 00:00:00'): Timestamp('2000-05-22 00:00:00'),
 Timestamp('2000-04-25 00:00:00'): Timestamp('2000-06-09 00:00:00'),
 Timestamp('2000-04-03 00:00:00'): Timestamp('2000-05-18 00:00:00'),
 Timestamp('2000-03-30 00:00:00'):

In [8]:
holdings["filing_deadline"] = holdings["date_holding"].apply(lambda x: filing_deadline_map[x])

In [9]:
holdings = holdings[holdings.date_filed <= holdings.filing_deadline]

# Filter for US Holdings, add ticker symbol, remove delisted symbols

In [9]:
# Ticker information for US holdings
ticker_meta = pd.read_csv('raw_data/ticker_meta.csv')

In [5]:
ticker_cusip_map = ticker_meta[(ticker_meta.exchange != "OTC") & (~ticker_meta.isDelisted)][["ticker","cusip"]]
ticker_cusip_map["cusip"] = ticker_cusip_map["cusip"].apply(lambda x: str(x).split(" "))
ticker_cusip_map = ticker_cusip_map.explode('cusip')
ticker_cusip_map = ticker_cusip_map.drop_duplicates()
ticker_cusip_map = ticker_cusip_map[ticker_cusip_map.cusip != "nan"]

In [6]:
us_holdings = holdings.merge(ticker_cusip_map, on="cusip", how="inner")

In [7]:
del holdings

In [8]:
# Remove ticker suffix to match with pricing data
us_holdings["ticker"] = us_holdings["ticker"].apply(lambda x: x.split(".")[0])

In [29]:
# Remove tickers that have no pricing data
close_data = pd.read_csv('raw_data/close_data.csv', index_col="date")
us_holdings = us_holdings[us_holdings.ticker.isin(close_data.columns)]

# Filter away data with put call flags

In [30]:
us_holdings_cleaned = us_holdings[us_holdings.putCall.isna()].drop(columns = "putCall")

In [31]:
del us_holdings

In [32]:
us_holdings_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69692583 entries, 0 to 74068588
Data columns (total 14 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   year                        int64  
 1   quarter                     int64  
 2   cik                         int64  
 3   company_name                object 
 4   date_filed                  object 
 5   date_holding                object 
 6   nameOfIssuer                object 
 7   titleOfClass                object 
 8   cusip                       object 
 9   value                       float64
 10  shrsOrPrnAmt_sshPrnamt      float64
 11  shrsOrPrnAmt_sshPrnamtType  object 
 12  filing_deadline             object 
 13  ticker                      object 
dtypes: float64(2), int64(3), object(9)
memory usage: 7.8+ GB


# Filter away data with incorrect formats

In [34]:
import re
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [35]:
share_types = ["SH", "Sh", "S", "SHS", "shs", "SHR", "sh", "Shares", "SHARES", "Shrs", "SHRS", "Shs", "Shr", "X", "x"]
# Filter only for holding of shares (Remove things like warrants which uses PRN principal value)
us_holdings_cleaned = us_holdings_cleaned[us_holdings_cleaned.shrsOrPrnAmt_sshPrnamtType.isin(share_types)].drop(columns = "shrsOrPrnAmt_sshPrnamtType")

In [36]:
# Remove rows with zero shares
us_holdings_cleaned = us_holdings_cleaned[us_holdings_cleaned.shrsOrPrnAmt_sshPrnamt != 0]

In [37]:
# Remove filings with reported holding date not at quarter end 
us_holdings_cleaned = us_holdings_cleaned[us_holdings_cleaned.date_holding.apply(lambda x: 
                                                                                 x.endswith(("03-31","06-30","09-30","12-31")))]

In [38]:
us_holdings_cleaned = us_holdings_cleaned[us_holdings_cleaned.shrsOrPrnAmt_sshPrnamt % 1 == 0]
us_holdings_cleaned["shrsOrPrnAmt_sshPrnamt"] = us_holdings_cleaned["shrsOrPrnAmt_sshPrnamt"].astype(int)
us_holdings_cleaned = us_holdings_cleaned[us_holdings_cleaned.shrsOrPrnAmt_sshPrnamt != 0]

C:\Users\ohste\anaconda3\envs\fyp\lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


# Filter for only single-stock holdings (Remove ETFs, ETNs etc.)

In [39]:
ticker_categories_filter = ticker_meta[ticker_meta.category.isin([x for x in ticker_meta.category.unique() 
                            if isinstance(x,str) and "Stock" in x and "Warrant" not in x])
                                      ].ticker.dropna().apply(lambda x: x.split(".")[0])

In [40]:
us_holdings_cleaned = us_holdings_cleaned[us_holdings_cleaned.ticker.isin(ticker_categories_filter)]

# Aggregate same holdings (ignoring voting rights and investment manager)

In [41]:
# Remove company name as can be mapped by cik
# Remove nameOfIssuer as can be mapped by ticker
# Remove cusip as using ticker to identify stock
# Remove value as using number of shares
# Remove filing deadline as can be mapped by date holding
agg_us_holdings = us_holdings_cleaned.drop(columns=
                         ["company_name","nameOfIssuer","titleOfClass", "year", "quarter", "cusip", "value", "filing_deadline"]
                        ).reindex(columns=
                                  ["cik","date_holding","date_filed","ticker","shrsOrPrnAmt_sshPrnamt"]
                                 ).groupby(
    ["cik","date_holding","date_filed","ticker"]
).shrsOrPrnAmt_sshPrnamt.sum().reset_index()

In [ ]:
agg_us_holdings.to_csv('cleaned_data/agg_us_holdings.csv', index=False)

# Add price and quarterly returns feature

In [ ]:
data = pd.read_csv('cleaned_data/agg_us_holdings.csv')
data["date_holding"] = pd.to_datetime(data["date_holding"])
data["date_filed"] = pd.to_datetime(data["date_filed"])

In [ ]:
close_data = pd.read_csv('raw_data/close_data.csv', index_col="date")
close_data.index = pd.to_datetime(close_data.index)
close_data_long = pd.melt(close_data, ignore_index=False, var_name="ticker", value_name="price").reset_index()

In [ ]:
tmp = pd.merge_asof(data.sort_values('date_holding'), 
              close_data_long.sort_values('date'), 
              left_on="date_holding", 
              right_on="date",
              by="ticker",
              tolerance=pd.Timedelta(days=4),
              direction='backward'
             )
tmp = tmp.rename(columns={"price": "buy_price"})

In [ ]:
tmp["date_selloff"] = tmp["date_holding"] + pd.tseries.offsets.QuarterEnd()
tmp = pd.merge_asof(tmp,
                    close_data_long.sort_values('date'),
                    left_on="date_selloff", 
                    right_on="date",
                    by="ticker",
                    tolerance=pd.Timedelta(days=4),
                    direction='backward'
                   )
tmp = tmp.rename(columns={"price": "sell_price"})

In [ ]:
tmp = tmp.drop(columns=["date_x", "date_y", "date_selloff"]).rename(columns = {"shrsOrPrnAmt_sshPrnamt": "num_shares"})
tmp["value"] = tmp["num_shares"] * tmp["buy_price"]
tmp["quarter_return"] = (tmp["sell_price"] - tmp["buy_price"]) / tmp["buy_price"]
tmp = tmp.dropna(subset=["buy_price","sell_price","value","quarter_return"])

In [ ]:
tmp.to_csv('cleaned_data/agg_us_holdings.csv', index=False)